In [23]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [24]:
import pandas as pd
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="prisjakt_tv_norway"
uname="root"
pwd="4156"


# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

In [25]:
base_url = "https://www.prisjakt.no"
url = "https://www.prisjakt.no/c/tv"
page_links = []
links = []
page_links.append(url)
for page in page_links:
    url = page
    i = requests.get(url).text
    soup=BeautifulSoup(i,'html.parser')
    pages = soup.find_all("li",{"class":"PaginationListItem-sc-11oisbu-1 bKXcXk"})
    for page in pages:
        link = page.find("a", {"data-test":"PaginationLink"})
        links.append(link)
        if link == None:
            links.remove(link)
    for l in links:
        pg = l.get('href')
        full_url = base_url+pg
        while full_url not in page_links:
            page_links.append(full_url)


print(f'Pages Found: {len(page_links)}')

Pages Found: 33


In [26]:
productlists = []
for link in page_links:
        k = requests.get(link).text
        soup=BeautifulSoup(k,'html.parser')
        productlist = soup.find_all("article",{"data-test":"ProductGridCard"})
        productlists.append(productlist)

In [27]:
items = [item for sublist in productlists for item in sublist]

In [28]:
productlinks = []
for item in items:
        link = item.find("a",{"data-test":"InternalLink"}).get('href')                 
        productlinks.append(base_url+link)

print(f'Number of Products Found: {len(productlinks)}')

Number of Products Found: 1446


In [29]:
df_prod_links = pd.DataFrame(productlinks, columns = ['Product Links'])
# Convert dataframe to sql table                                   
df_prod_links.to_sql('productlinks', engine, if_exists='replace', index=False)

1446

In [30]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [31]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_24896\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [32]:
property_list = []
for link in productlinks:
    url = link+"#properties"
    driver.get(url)
    try:
        element1 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//body/div[@id='root']/div[@class='App-sc-2fu3f8-0 ljZBoE']/section[@class='Main-sc-2fu3f8-1 hnCjSa']/div[@class='Content-sc-2fu3f8-2 hybPGh']/div[@class='PageContent-sc-1wgu331-5 dfQdce']/div[@class='Wrapper-sc-1wgu331-1 gPsMfp']/div[@class='Content-sc-1wgu331-0 gMPJqu']/div[@class='ProductPage-sc-mnx6sq-0 iMqgaO']/div/div[@id='#properties']/div[@class='PageSectionWrapper-sc-o2pbvx-0 kQdrfN']/section[@class='PageSectionContent-sc-o2pbvx-1 hbBYkf']/section[@data-test='PropertiesTabContent']/div[@class='StyledList--ji0zot McirA StyledPanel--17i5iqi ffQfPu']/div[@data-test-type='product-info']/div[@class='StyledViewport-sc-7zjdbj-0 iDfoDl']/section[1]/div[1]/div[1]"))
        )
        element1.click()
    except:
        element1 = None
    page_source =driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    specs = soup.find_all("section", {"class":"Section-sc-1jnk5ag-1 kLbzuK"})
    prop_names = []
    prop_vals = []
    for sect in specs:
        sections = sect.find_all("div", {"class", "Row-sc-1jnk5ag-3 jQedgK"})
        for section in sections:
            properties = section.find_all("div", {"class":"Column-sc-1jnk5ag-2 bIOAXd"})
            for property in properties:
                property_name = property.find("span", {"class":"Text--bzqghn wmwsE bodysmalltext PropertyName-sc-1jnk5ag-5 hEdIXF"})
                property_value = property.find("span", {"class":"Text--bzqghn ddqBq bodysmalltext PropertyValue-sc-1jnk5ag-6 etcKHO"})
                if property_name !=None:
                    prop_names.append(property_name.text.replace('\n',""))
                elif property_value !=None:
                    prop_vals.append(property_value.text.replace('\n',""))
    properties = {}
    for name in prop_names:
        for val in prop_vals:
            properties[name] = val
            prop_vals.remove(val)
            break
    property_list.append(properties)


In [35]:
def get_prices():
        try:
        #  click on cookie banner
            banner_click = driver.find_element(By.CSS_SELECTOR, "div.BannerCookieBanner div.BannerContentCookieBanner div.ButtonsWrapperCookieBanner.AcceptButtonWrapperCookieBanner button.AcceptButtonCookieBanner.ButtonCookieBanner span.TextWrapperCookieBanner > span:nth-child(1)")
            banner_click.click()
        except:
            banner_click = None
        try:
                #click to reveal more items
            reveal_click = driver.find_element(By.CSS_SELECTOR, "div.App-sc-2fu3f8-0.ljZBoE section.Main-sc-2fu3f8-1.hnCjSa:nth-child(5) div.Content-sc-2fu3f8-2.hybPGh div.PageContent-sc-1wgu331-5.dfQdce div.Wrapper-sc-1wgu331-1.gPsMfp div.Content-sc-1wgu331-0.gMPJqu div.ProductPage-sc-mnx6sq-0.iMqgaO div.SectionWrapper-sc-ia0zhw-0.bqrwQK:nth-child(1) div.PageSectionWrapper-sc-o2pbvx-0.gHOcYt.StyledPageSection-sc-1w3nqup-0.UUqee:nth-child(1) section.PageSectionContent-sc-o2pbvx-1.euuGdp section:nth-child(1) div.PriceListPanelWrapper-sc-1htx0wh-0.lkPiWw div.PricePanel-sc-wkzg9v-1.kSsiTe div:nth-child(3) > button.BaseButton--uexw3i.cVdzxy.textbutton")
            reveal_click.click()
        except:
            reveal_click = None

        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        offers  = soup.find("ul", {"class":"PriceList-sc-wkzg9v-0 fbrkVc"})
        price_list = []
        for offer in offers:
            offer_prices  = offer.find_all("div", {"class":"PriceWrapper-sc-k40pbc-4 OYetQ"})
            price_list.append(offer_prices)

        price_list = [price for sublist in price_list for price in sublist]
        for i in range(0, len(price_list)):
            price_list[i] = price_list[i].text.replace('\n',"")

        for i in range(0, len(price_list)):
            price_list[i] = re.sub('[\xa0]', '',price_list[i])
                #price_list[i] = re.sub('[kr]', '', price_list[i])
            price_list[i] = re.sub("[^0-9]", "", price_list[i])
                #price_list[i]= price_list[i].replace(r'[()]',"")

        for i in range(0, len(price_list)):
            price_list[i] = int(price_list[i])
        no_offers = len(price_list)
        avg_price =  "{:.2f}".format(mean(price_list))

        return no_offers, avg_price

In [36]:
data = []
product_scraped = 1
for link in productlinks:
    from datetime import datetime
    f = requests.get(link,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')
    driver.get(link)

    try:
        category = "TV"
    except:
        category = None
    try:
        name = hun.find("div",{"data-test":"ProductTitle"}).text.replace('\n',"")
    except:
        name = None
    try:
        avg_price = get_prices()[1]
    except:
        avg_price = None
    try:
        offs = get_prices()[0]
    except:
        offs = None
    try:
        country = "NO"
    except:
        country = None
    try:
        currency = "NOK"
    except:
        currency = None
    try:
        rank = hun.find("span",{"class":"Text--bzqghn kBBoFI titlesmalltext StyledCaptionText--7mhtyj fxXtPM"}).text.replace('\n',"")
    except:
        rank = None
    try:
        rating = hun.find("span",{"class":"RateNumber-sc-14ktvqu-5 ifhFUL"}).text.replace('\n',"")
    except:
        rating = None
    try:
        reviews = hun.find("span",{"class":"Counter-sc-14ktvqu-0 bORxLl"}).text.replace('\n',"")
    except:
        reviews = None
    try:
        description = hun.find("p",{"class":"ProductInfo-sc-46oo5c-0 bRxlhS"}).text.replace('\n',"")
    except:
        description = None
    #try:
        #specifications = get_specs(prod_specifications)
    #except:
        #specifications = None

    try:
        scrape_link = link
        
    except:
        scrape_link = None
    
    try:
        now = datetime.now()
        date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    except:
        date_scraped = None


    tv = {"Country Code":country, "Category":category, "Name":name, "Avg Price": avg_price, "Currency":currency, "Nb of Offers":offs, "Rank":rank, "Rating":rating, "Reviews":reviews, "Description":description, \
        "scrape_link":scrape_link, "date_scraped":date_scraped}
    
    data.append(tv)
    
    print(f'product scraped: {product_scraped, name}')
    product_scraped = product_scraped + 1
    

product scraped: (1, 'LG OLED65C1 65" 4K Ultra HD (3840x2160) OLED Smart TV')
product scraped: (2, 'LG OLED55C1 55" 4K Ultra HD (3840x2160) OLED Smart TV')
product scraped: (3, 'LG OLED65B1 65" 4K Ultra HD (3840x2160) OLED Smart TV')
product scraped: (4, 'Samsung The Frame QE55LS03A 55" 4K Ultra HD (3840x2160) QLED Smart TV')
product scraped: (5, 'Samsung Neo QLED QE65QN90A 65" 4K Ultra HD (3840x2160) Smart TV')
product scraped: (6, 'Samsung The Frame QE65LS03A 65" 4K Ultra HD (3840x2160) QLED Smart TV')
product scraped: (7, 'Philips 70PUS9006 70" 4K Ultra HD (3840x2160) LCD Smart TV')
product scraped: (8, 'Samsung QLED QE85Q80A 85" 4K Ultra HD (3840x2160) Smart TV')
product scraped: (9, 'Samsung QLED QE65Q80B 65" 4K Ultra HD (3840x2160) Smart TV')
product scraped: (10, 'LG 75UP7500 75" 4K Ultra HD (3840x2160) LCD Smart TV')
product scraped: (11, 'Samsung QLED QE50Q60A 50" 4K Ultra HD (3840x2160) Smart TV')
product scraped: (12, 'Philips 58PUS9006 58" 4K Ultra HD (3840x2160) LCD Smart 

In [37]:
df_prop = pd.DataFrame.from_dict(property_list)
df_prop.head()

,Produsenter,Bildestørrelse,Oppløsning,Skjermteknologi,HDR,Smart TV,Lanseringsår,Pris/tommer,Produktnavn,Kategori,...,Energiklasse (HDR),Strømforbruk operativ (min-maks),Strømforbruk standby/hvilemodus,Tilkobling for 9V/12V/14V,Bakgrunnsbelysning,Diodeplassering,Teknologier,Paneltype,Brilleteknologi,Medfølgende briller
0,LG,65 Tommer,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,215.23 kr/tomme,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",TV,...,G,126 - 222 W,0.5 W,Nei,NaN,NaN,NaN,NaN,NaN,NaN
1,LG,55 Tommer,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,218 kr/tomme,"LG OLED55C1 55"" 4K Ultra HD (3840x2160) OLED S...",TV,...,G,106 W,0.5 W,Nei,NaN,NaN,NaN,NaN,NaN,NaN
2,LG,65 Tommer,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,161.38 kr/tomme,"LG OLED65B1 65"" 4K Ultra HD (3840x2160) OLED S...",TV,...,G,122 W,0.5 W,Nei,NaN,NaN,NaN,NaN,NaN,NaN
3,Samsung,55 Tommer,4K Ultra HD (3840x2160),LCD,Ja,Ja,2021,161.6 kr/tomme,"Samsung The Frame QE55LS03A 55"" 4K Ultra HD (3...",TV,...,G,170 W,0.5 W,Nei,LED (lysdiode),,Quantum dot,,NaN,NaN
4,Samsung,65 Tommer,4K Ultra HD (3840x2160),LCD,Ja,Ja,2021,338.31 kr/tomme,"Samsung Neo QLED QE65QN90A 65"" 4K Ultra HD (38...",TV,...,G,285 W,0.5 W,Nei,LED (lysdiode),Bakbelyst (Back),"Quantum dot, Lokal dimming, Mini-LED",IPS,NaN,NaN


In [38]:
df = pd.DataFrame.from_dict(data)
df.tail()

,Country Code,Category,Name,Avg Price,Currency,Nb of Offers,Rank,Rating,Reviews,Description,scrape_link,date_scraped
1441,NO,TV,"Toshiba 50QA7D63DG 50"" 4K Ultra HD (3840x2160)...",6490.00,NOK,1.0,Rangering 1647,None,(1),Den billigste prisen for Toshiba 50QA7D63DG 50...,https://www.prisjakt.no/product.php?p=7005951,23/08/2022 10:06:37
1442,NO,TV,"Toshiba 65QA7D63DG 65"" 4K Ultra HD (3840x2160)...",8990.00,NOK,1.0,Rangering 1649,None,(1),Den billigste prisen for Toshiba 65QA7D63DG 65...,https://www.prisjakt.no/product.php?p=7005956,23/08/2022 10:06:39
1443,NO,TV,"Toshiba 43QA7D63DG 43"" 4K Ultra HD (3840x2160)...",5990.00,NOK,1.0,Rangering 1650,None,(1),Den billigste prisen for Toshiba 43QA7D63DG 43...,https://www.prisjakt.no/product.php?p=7005958,23/08/2022 10:06:40
1444,NO,TV,"Philips 43PUS8107 43"" 4K Ultra HD (3840x2160) ...",5499.00,NOK,1.0,Rangering 1651,None,(1),"Den billigste prisen for Philips 43PUS8107 43""...",https://www.prisjakt.no/product.php?p=7005959,23/08/2022 10:06:41
1445,NO,TV,"JVC LT-58VU3000 58"" 4K Ultra HD (3840x2160) LC...",6781.00,NOK,2.0,Rangering 1652,None,(1),"Den billigste prisen for JVC LT-58VU3000 58"" 4...",https://www.prisjakt.no/product.php?p=7005973,23/08/2022 10:06:43


In [39]:
df.columns

Index(['Country Code', 'Category', 'Name', 'Avg Price', 'Currency',
       'Nb of Offers', 'Rank', 'Rating', 'Reviews', 'Description',
       'scrape_link', 'date_scraped'],
      dtype='object')

In [40]:
df = pd.concat([df, df_prop.reindex(df.index)], axis=1)
#gen_spec_df = df["Generalspecifications"].apply(pd.Series)
#df = pd.concat([df, gen_spec_df.reindex(df.index)], axis=1)
df_c = df.copy()
# Remove specifications column
#df.drop(['specifications'], axis=1, inplace=True)
df.shape

(1446, 98)

In [45]:
#df2 = df.reset_index().rename(columns={'index':'id'})
df_long = df.set_index(['Country Code', 'Produsenter','Kategori', 'Name', 'Nb of Offers', 'Lanseringsår', 'Avg Price', 'Currency', 'scrape_link', 'date_scraped']).stack().reset_index()

In [46]:
# Convert dataframe to sql table                                   
df_long.to_sql('data_table',  engine, if_exists='replace', index=False)

98003

In [47]:
df.to_csv("data.csv")

    prod_specifications = hun.find_all("div",{"class":"Column-sc-1jnk5ag-2 bIOAXd"})
    def get_specs(prod_specifications):
        nam = []
        val = []
        for spec in prod_specifications:
            for prop in spec.find_all("span", {"class":"Text--bzqghn wmwsE bodysmalltext PropertyName-sc-1jnk5ag-5 hEdIXF"}):
                prop_n  = prop.get_text()
                nam.append(prop_n)
            for prop in spec.find_all("span", {"class":"Text--bzqghn ddqBq bodysmalltext PropertyValue-sc-1jnk5ag-6 etcKHO"}):
                prop_v  = prop.get_text()
                val.append(prop_v)
        specs = {nam[i]: val[i] for i in range(len(nam))}
        return(specs)

In [44]:
df_long.to_csv('df_long.csv')